In [1]:
from pathlib import Path
import streamlit as st
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain  
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
import time

# Definindo o caminho para os arquivos usando a lib PathLib que serve para manipulação de arquivos e diretorios
FILES_PATH = Path('/Users/maelsonmarquesdelima/Library/CloudStorage/OneDrive-Pessoal/Documentos/ESTUDAR/DATA_SCIENCE_PYTHON/AssistenteContexto/test.ipynb').parent / "files"
MODEL_NAME = 'gpt-3.5-turbo-0125'

def import_documents():
    documents = []
    for arquivo in FILES_PATH.glob("*.pdf"):
        loader = PyPDFLoader(str(arquivo)) #traz o caminho do arquivo
        documents_loader = loader.load()
        documents.extend(documents_loader)
    return documents

def split_documents(documents):
    recur_spliter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=250,
        separators= ["\n\n", "\n", ".", "!", "?", ";", ":", "\t"]
    )
    documents = recur_spliter.split_documents(documents)

    for i, doc in enumerate(documents): # interador que faz 
        doc.metadata['source'] = doc.metadata['source'].split("/")[-1]#
        doc.metadata['doc_id'] = i
    return documents

def create_vector_store(documents):
    embedding_model = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(
        documents = documents,
        embedding = embedding_model,
    )
    return vector_store

def criar_chain_conversa(vector_store):
    chat = ChatOpenAI(model=MODEL_NAME)
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key='chat_history',
        output_key='answer',
    )
    retriever = vector_store.as_retriever()
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        memory=memory,
        retriever=retriever,
        return_source_documents=True,
        verbose=True
    )
    return chat_chain


In [2]:

documents = import_documents()
documents = split_documents(documents)
vector_store = create_vector_store(documents)
chain = criar_chain_conversa(vector_store)


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
/var/folders/xb/g2pjwx_s2pn093t44p9_dgqm0000gn/T/ipykernel_97953/2150047210.py:47: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
chain.invoke({'question': 'Qual a importância da análise de dados?'})
